### RAG using Pinecone Vector Store

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# Embeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings

os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
embeddings_google = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
embeddings_google

GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x117efe150>, model='models/embedding-001', task_type=None, google_api_key=SecretStr('**********'), credentials=None, client_options=None, transport=None, request_options=None)

In [4]:
len(embeddings_google.embed_query("Hello AI"))

768

In [5]:
# Google LLM
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [6]:
# Prompt
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")
prompt.messages

/Users/I572648/Library/CloudStorage/OneDrive-SAPSE/Desktop/Git/GenAI/.venv/lib/python3.11/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]

In [9]:
# Pinecone Vector store
from pinecone import Pinecone, ServerlessSpec

pinecone_api_key = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

index_name = "agenticindex2"
if not pc.has_index(index_name):
    print(f"Creating index {index_name}...")
    pc.create_index(
        name=index_name, 
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )


Creating index agenticindex2...


In [11]:
from langchain_pinecone import PineconeVectorStore

index = pc.Index(index_name)
vectorstore = PineconeVectorStore(
    index=index,
    embedding=embeddings_google
)

In [12]:
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

In [13]:
from uuid import uuid4

# universal identification number
uuids = [str(uuid4()) for _ in range(len(documents))]

vectorstore.add_documents(documents=documents, ids=uuids)

['2f955fda-f49f-476f-80d4-3b2892815443',
 'a87a0375-c3f0-4798-a9d9-2687580f8a55',
 '567cffcc-d933-4358-b243-0e9d96eb37f4',
 'a6801d17-7351-4098-8922-2293cbb7e7c7',
 '04e6f503-8464-4a4b-8137-c7bd7e325881',
 'e4de4a0c-b1a1-41c3-ac22-0fe989d1fddb',
 '905537b0-198b-46ef-a38c-7cb0509c46d8',
 '6b6d7e39-de5b-4a42-acff-5b63a6ad28e7',
 '4284c821-55c3-45e7-abf5-919712f405d9',
 '30c2a668-caae-4193-85a6-3adb20bd53c0']

In [14]:
retriever = vectorstore.as_retriever(search_type = "similarity_score_threshold", search_kwargs={"k": 3, "score_threshold": 0.8})
retriever

VectorStoreRetriever(tags=['PineconeVectorStore', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x1231dac90>, search_type='similarity_score_threshold', search_kwargs={'k': 3, 'score_threshold': 0.8})

In [15]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [17]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

In [18]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [19]:
rag_chain.invoke("What is LangGraph?")

"LangGraph is a framework designed for creating stateful, agentic applications.  It's described as excellent for building such projects.  More details about its specific features are not provided in the context."